In [1]:
import numpy as np
from typing import List, Tuple
import os

def generate_flowstar_model(
    Ac: np.ndarray,  # Continuous-time A matrix
    Bc: np.ndarray,  # Continuous-time B matrix
    Cc: np.ndarray,  # Continuous-time C matrix
    Dc: np.ndarray,  # Continuous-time D matrix
    K: np.ndarray,   # Discrete-time controller gain
    L: np.ndarray,   # Discrete-time observer gain
    num_locations: int,  # Number of locations in the hybrid automaton
    length: int,     # Length of the (m,k)-weakly hard pattern
    miss_count: int, # Number of allowed misses in the pattern
    output_file: str, # Output .model file path
    initial_states: List[Tuple[float, float]] = None,  # List of (min, max) for each state
    input_bounds: List[Tuple[float, float]] = None,    # List of (min, max) for each input
    sampling_time: float = 0.01,  # Sampling time for the discrete controller
    time_horizon: float = 10.0    # Total simulation time
) -> None:
    """
    Generate a Flowstar model file for a hybrid system with weakly-hard constraints.
    
    Parameters:
    -----------
    Ac, Bc, Cc, Dc : numpy.ndarray
        System matrices in continuous time
    K : numpy.ndarray
        Discrete-time controller gain
    L : numpy.ndarray
        Discrete-time observer gain
    num_locations : int
        Number of locations in the hybrid automaton
    length : int
        Length of the (m,k)-weakly hard pattern
    miss_count : int
        Number of allowed misses in the pattern
    output_file : str
        Path to save the generated .model file
    initial_states : List[Tuple[float, float]], optional
        Initial state bounds for each state variable
    input_bounds : List[Tuple[float, float]], optional
        Input bounds for each input
    sampling_time : float, optional
        Sampling time of the discrete controller
    time_horizon : float, optional
        Total simulation time
    """

In [2]:
def generate_state_vars(n_states: int) -> str:
    """Generate state variable declarations"""
    return '\n'.join([f'state var x{i}' for i in range(n_states)])

def generate_mode_vars(n_modes: int) -> str:
    """Generate mode variable declarations"""
    return 'discrete var mode, pattern_pos\n'

def matrix_to_flowstar(matrix: np.ndarray, prefix: str = '') -> str:
    """Convert a numpy matrix to Flowstar format"""
    rows, cols = matrix.shape
    lines = []
    for i in range(rows):
        terms = []
        for j in range(cols):
            val = matrix[i, j]
            if abs(val) > 1e-10:  # Ignore very small values
                if j == 0:
                    terms.append(f'{val}')
                else:
                    terms.append(f'{val}*{prefix}{j}')
        lines.append(' + '.join(terms) if terms else '0')
    return '\n'.join(lines)

def generate_init_section(initial_states: List[Tuple[float, float]]) -> str:
    """Generate the init section"""
    conditions = []
    for i, (min_val, max_val) in enumerate(initial_states):
        conditions.append(f'x{i} in [{min_val}, {max_val}]')
    return 'init\n{\n\tmode == 0\n\t' + '\n\t'.join(conditions) + '\n}'

def generate_continuous_dynamics(Ac: np.ndarray, Bc: np.ndarray) -> str:
    """Generate continuous dynamics equations"""
    n_states = Ac.shape[0]
    dynamics = []
    
    # Convert continuous-time state equations
    state_eqs = Ac @ np.array([f'x{i}' for i in range(n_states)]).reshape(-1, 1)
    input_terms = Bc @ np.array([f'u{i}' for i in range(Bc.shape[1])]).reshape(-1, 1)
    
    for i in range(n_states):
        dynamics.append(f"x{i}' = {state_eqs[i, 0]}")
        if Bc.shape[1] > 0:
            dynamics.append(f" + {input_terms[i, 0]}")
            
    return '\n'.join(dynamics)

In [3]:
def generate_flowstar_model(
    Ac: np.ndarray,
    Bc: np.ndarray,
    Cc: np.ndarray,
    Dc: np.ndarray,
    K: np.ndarray,
    L: np.ndarray,
    num_locations: int,
    length: int,
    miss_count: int,
    output_file: str,
    initial_states: List[Tuple[float, float]] = None,
    input_bounds: List[Tuple[float, float]] = None,
    sampling_time: float = 0.01,
    time_horizon: float = 10.0
) -> None:
    n_states = Ac.shape[0]
    n_inputs = Bc.shape[1]
    
    if initial_states is None:
        initial_states = [(0.0, 0.0)] * n_states
    
    if input_bounds is None:
        input_bounds = [(-1.0, 1.0)] * n_inputs
        
    # Generate the model file content
    model_content = []
    
    # Variable declarations
    model_content.append("# State variables")
    model_content.append(generate_state_vars(n_states))
    model_content.append("\n# Mode variables")
    model_content.append(generate_mode_vars(num_locations))
    
    # Settings
    model_content.append("\nsetting")
    model_content.append("{")
    model_content.append(f"\tfixed steps {sampling_time}")
    model_content.append(f"\ttime {time_horizon}")
    model_content.append("\tremainder estimation 1e-4")
    model_content.append("\tidentity precondition")
    model_content.append("\tgnuplot octagon x0,x1")
    model_content.append("\toutput out")
    model_content.append("}")
    
    # Initial conditions
    model_content.append("\n" + generate_init_section(initial_states))
    
    # Generate locations (modes)
    for loc in range(num_locations):
        model_content.append(f"\n# Location {loc}")
        model_content.append(f"loc l{loc}")
        model_content.append("{")
        
        # Continuous dynamics
        model_content.append("\tdynamic")
        model_content.append("\t{")
        dynamics = generate_continuous_dynamics(Ac, Bc)
        for line in dynamics.split('\n'):
            model_content.append(f"\t\t{line}")
        model_content.append("\t}")
        
        # Invariant conditions
        model_content.append("\tinv")
        model_content.append("\t{")
        model_content.append(f"\t\tmode == {loc}")
        model_content.append("\t}")
        
        model_content.append("}")
        
    # Generate transitions
    for loc in range(num_locations):
        next_loc = (loc + 1) % num_locations
        model_content.append(f"\n# Transition from l{loc} to l{next_loc}")
        model_content.append(f"trans")
        model_content.append("{")
        model_content.append(f"\tfrom: l{loc}")
        model_content.append(f"\tto: l{next_loc}")
        model_content.append("\tguard { pattern_pos == 0 }")
        model_content.append("\treset {")
        model_content.append(f"\t\tmode' := {next_loc}")
        model_content.append("\t\tpattern_pos' := pattern_pos + 1")
        model_content.append("\t}")
        model_content.append("}")
    
    # Write the model file
    with open(output_file, 'w') as f:
        f.write('\n'.join(model_content))

In [4]:
# Example usage
if __name__ == "__main__":
    # Example system matrices (2-state system)
    Ac = np.array([
        [0, 1],
        [-2, -3]
    ])
    
    Bc = np.array([
        [0],
        [1]
    ])
    
    Cc = np.array([
        [1, 0]
    ])
    
    Dc = np.array([[0]])
    
    # Example controller and observer gains
    K = np.array([[1, 2]])
    L = np.array([[0.5], [1.0]])
    
    # Weakly-hard parameters
    num_locations = 4  # Number of locations in automaton
    length = 5        # Length of pattern
    miss_count = 2    # Number of allowed misses
    
    # Initial state bounds
    initial_states = [(-1, 1), (-1, 1)]  # For x0 and x1
    
    # Generate the model file
    generate_flowstar_model(
        Ac=Ac,
        Bc=Bc,
        Cc=Cc,
        Dc=Dc,
        K=K,
        L=L,
        num_locations=num_locations,
        length=length,
        miss_count=miss_count,
        output_file="example_model.model",
        initial_states=initial_states,
        sampling_time=0.01,
        time_horizon=10.0
    )
    
print("Model file has been generated!")

UFuncTypeError: ufunc 'matmul' did not contain a loop with signature matching types (dtype('int64'), dtype('<U2')) -> None